In [38]:
# 1. Import Dependencies
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [39]:
# 2. Make Detections
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [40]:
# # 3. Determining Joints
# cap = cv2.VideoCapture(0)

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()

#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         results = pose.process(image)

#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         try:
#             landmarks = results.pose_landmarks.landmark
#             print(landmarks)
#         except:
#             pass

#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()
# len(landmarks)
# for lndmrk in mp_pose.PoseLandmark:
#     print(lndmrk)
# landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
# landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
# landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

In [43]:

def calculate_angle(a, b, c):
    a = np.array(a)  
    b = np.array(b)  
    c = np.array(c)  
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

def detect_pose(hip_angle, knee_angle, ankle_y, hip_y, active_pose):
    if active_pose == "Standing" and hip_angle > 160 and knee_angle > 160:
        return "Standing"
    elif active_pose == "Sitting" and hip_angle < 140 and knee_angle < 140:
        return "Sitting"
    elif active_pose == "Jumping" and hip_angle > 150 and knee_angle > 150 and ankle_y < hip_y:
        return "Jumping"
    elif active_pose == "Walking" and hip_angle > 140 and knee_angle < 160:
        return "Walking"
    return "Unknown Pose"

def draw_buttons(image):
    button_config = {
        "Standing": (10, 50),
        "Sitting": (10, 100),
        "Walking": (10, 150),
        "Jumping": (10, 200)
    }
    for text, pos in button_config.items():
        cv2.rectangle(image, (pos[0], pos[1] - 40), (pos[0] + 150, pos[1]), (245, 117, 16), -1)
        cv2.putText(image, text, (pos[0] + 10, pos[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return button_config

def check_button_click(mouse_x, mouse_y, button_config):
    for pose, (x, y) in button_config.items():
        if x <= mouse_x <= x + 150 and y - 40 <= mouse_y <= y:
            return pose
    return None

cap = cv2.VideoCapture(0)

active_pose = None  

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        
        button_config = draw_buttons(image)

        try:

            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            hip_angle = calculate_angle(shoulder, hip, knee)
            knee_angle = calculate_angle(hip, knee, ankle)

            pose_status = detect_pose(hip_angle, knee_angle, ankle[1], hip[1], active_pose)

        except:
            pose_status = "Unknown Pose"

        cv2.rectangle(image, (200, 0), (600, 50), (0, 0, 0), -1)
        cv2.putText(image, f'Pose: {pose_status if active_pose else "Select Pose"}', (210, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                 )               

        cv2.imshow('Mediapipe Feed', image)


        def on_mouse(event, x, y, flags, param):
            global active_pose
            if event == cv2.EVENT_LBUTTONDOWN:
                selected_pose = check_button_click(x, y, button_config)
                if selected_pose:
                    active_pose = selected_pose

        cv2.setMouseCallback('Mediapipe Feed', on_mouse)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()